#### Engenharia de dados
**Nome**: MÁRCIO DE LIMA LOSS PUGNAL  
**Código Aluno**: 4052025000363  
**Curso**: Pós CIÊNCIA DE DADOS E ANALYTICS  
**Dataset**: [Board-Games] [fonte: https://www.kaggle.com/datasets/andrewmvd/board-games ]

#### Importação das bibliotecas, coleta e carga de dados.

In [0]:
# Importação de bibliotecas

# Manipulação de dados
import pandas as pd
import requests
#from io import StringIO

# Spark para processamento distribuído
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import (
    col,                          # Seleção e manipulação de colunas
    monotonically_increasing_id,  # Gerar IDs únicos
    regexp_replace,               # Substituir padrões usando regex
    explode,                      # Expandir arrays em múltiplas linhas
    split                         # Dividir strings em arrays
)

# Visualização de dados
import matplotlib.pyplot as plt               # Criação de gráficos básicos em Python
from matplotlib.ticker import FuncFormatter   # Formatar valores nos eixos (ex.: porcentagem, moeda)
from matplotlib.ticker import MultipleLocator # Definir espaçamento fixo entre marcas dos eixos
import seaborn as sns                         # Para gráficos estatísticos mais elaborados e prontos

In [0]:
# Coleta e carga de dados

# URL do dataset
file_path = "/Workspace/Users/marciopug@gmail.com/mvp-engenharia-dados/bgg_dataset.csv"

# Ler direto com pandas
df_dados = pd.read_csv(file_path, delimiter=';')

# Converter para Spark DataFrame
df_tabuleiro = spark.createDataFrame(df_dados)

# Informações do dataset carregado
print("Dataset carregado com sucesso!")
print(f"Total de registros: {df_tabuleiro.count()}")
print(f"Total de colunas..: {len(df_tabuleiro.columns)}")

Dataset carregado com sucesso!
Total de registros: 20343
Total de colunas..: 14


#### Análise exploratória inicial

In [0]:
# Visualizar os dados
display(df_tabuleiro.limit(5))

ID,Name,Year Published,Min Players,Max Players,Play Time,Min Age,Users Rated,Rating Average,BGG Rank,Complexity Average,Owned Users,Mechanics,Domains
174430.0,Gloomhaven,2017.0,1,4,120,14,42055,"8,79",1,"3,86",68323.0,"Action Queue, Action Retrieval, Campaign / Battle Card Driven, Card Play Conflict Resolution, Communication Limits, Cooperative Game, Deck Construction, Deck Bag and Pool Building, Grid Movement, Hand Management, Hexagon Grid, Legacy Game, Modular Board, Once-Per-Game Abilities, Scenario / Mission / Campaign Game, Simultaneous Action Selection, Solo / Solitaire Game, Storytelling, Variable Player Powers","Strategy Games, Thematic Games"
161936.0,Pandemic Legacy: Season 1,2015.0,2,4,60,13,41643,"8,61",2,"2,84",65294.0,"Action Points, Cooperative Game, Hand Management, Legacy Game, Point to Point Movement, Set Collection, Trading, Variable Player Powers","Strategy Games, Thematic Games"
224517.0,Brass: Birmingham,2018.0,2,4,120,14,19217,"8,66",3,"3,91",28785.0,"Hand Management, Income, Loans, Market, Network and Route Building, Score-and-Reset Game, Tech Trees / Tech Tracks, Turn Order: Stat-Based, Variable Set-up",Strategy Games
167791.0,Terraforming Mars,2016.0,1,5,120,12,64864,"8,43",4,"3,24",87099.0,"Card Drafting, Drafting, End Game Bonuses, Hand Management, Hexagon Grid, Income, Set Collection, Solo / Solitaire Game, Take That, Tile Placement, Turn Order: Progressive, Variable Player Powers",Strategy Games
233078.0,Twilight Imperium: Fourth Edition,2017.0,3,6,480,14,13468,"8,7",5,"4,22",16831.0,"Action Drafting, Area Majority / Influence, Area-Impulse, Dice Rolling, Follow, Grid Movement, Hexagon Grid, Modular Board, Trading, Variable Phase Order, Variable Player Powers, Voting","Strategy Games, Thematic Games"


In [0]:
# Exibir tipos de dados
print("Tipos de dados:")
display(spark.createDataFrame(df_tabuleiro.dtypes, ["Atributo", "Tipo"]))

Tipos de dados:


Atributo,Tipo
ID,double
Name,string
Year Published,double
Min Players,bigint
Max Players,bigint
Play Time,bigint
Min Age,bigint
Users Rated,bigint
Rating Average,string
BGG Rank,bigint


In [0]:
# Mostrar as primeiras 5 linhas    
df_tabuleiro.select("ID", "Year Published", "Rating Average", "Complexity Average", "Owned Users").show(5)

+--------+--------------+--------------+------------------+-----------+
|      ID|Year Published|Rating Average|Complexity Average|Owned Users|
+--------+--------------+--------------+------------------+-----------+
|174430.0|        2017.0|          8,79|              3,86|    68323.0|
|161936.0|        2015.0|          8,61|              2,84|    65294.0|
|224517.0|        2018.0|          8,66|              3,91|    28785.0|
|167791.0|        2016.0|          8,43|              3,24|    87099.0|
|233078.0|        2017.0|           8,7|              4,22|    16831.0|
+--------+--------------+--------------+------------------+-----------+
only showing top 5 rows


#### Pré-processamentos dos dados

In [0]:
# Converter colunas de double para int
df_tabuleiro = df_tabuleiro.withColumn("ID", F.coalesce(F.col("ID"), F.lit(0)).cast("int"))
df_tabuleiro = df_tabuleiro.withColumn("Year Published", F.coalesce(F.col("Year Published"), F.lit(0)).cast("int"))
df_tabuleiro = df_tabuleiro.withColumn("Owned Users", F.coalesce(F.col("Owned Users"), F.lit(0)).cast("int"))

# Substituir vírgulas por ponto em decimais
df_tabuleiro = df_tabuleiro.withColumn("Rating Average", regexp_replace(col("Rating Average"), ",", ".").cast("double"))
df_tabuleiro = df_tabuleiro.withColumn("Complexity Average", regexp_replace(col("Complexity Average"), ",", ".").cast("double"))


In [0]:
# Converter outras colunas para int
colunas_int = ["Min Players", "Max Players", "Play Time", "Min Age", "Users Rated", "BGG Rank"]

# Percorrer as colunas e converter para int
for coluna in colunas_int:
    df_tabuleiro = df_tabuleiro.withColumn(coluna, F.coalesce(F.col(coluna), F.lit(0)).cast("int"))

In [0]:
# Visualizar amostra dos dados
df_tabuleiro.select("ID", "Year Published", "Rating Average", "Complexity Average", "Owned Users").show(5)

+------+--------------+--------------+------------------+-----------+
|    ID|Year Published|Rating Average|Complexity Average|Owned Users|
+------+--------------+--------------+------------------+-----------+
|174430|          2017|          8.79|              3.86|      68323|
|161936|          2015|          8.61|              2.84|      65294|
|224517|          2018|          8.66|              3.91|      28785|
|167791|          2016|          8.43|              3.24|      87099|
|233078|          2017|           8.7|              4.22|      16831|
+------+--------------+--------------+------------------+-----------+
only showing top 5 rows


In [0]:
# Verificar valores ausentes
dados = []
for c in df_tabuleiro.columns:
    ausentes = df_tabuleiro.filter(col(c).isNull()).count()
    presentes = df_tabuleiro.filter(col(c).isNotNull()).count()
    dados.append((c, presentes, ausentes))

# Cria DataFrame com os resultados
df_resumo = spark.createDataFrame(dados, ["Atributos", "Valores Presentes", "Valores Ausentes"])
df_resumo.show(truncate=False)

+------------------+-----------------+----------------+
|Atributos         |Valores Presentes|Valores Ausentes|
+------------------+-----------------+----------------+
|ID                |20343            |0               |
|Name              |20343            |0               |
|Year Published    |20343            |0               |
|Min Players       |20343            |0               |
|Max Players       |20343            |0               |
|Play Time         |20343            |0               |
|Min Age           |20343            |0               |
|Users Rated       |20343            |0               |
|Rating Average    |20343            |0               |
|BGG Rank          |20343            |0               |
|Complexity Average|20343            |0               |
|Owned Users       |20343            |0               |
|Mechanics         |18745            |1598            |
|Domains           |10184            |10159           |
+------------------+-----------------+----------

In [0]:
# Tratamento de missing (valores ausentes)

# Eliminando linhas que tenham algum valor missing.
df_tabuleiro = df_tabuleiro.dropna(how='any')

dados = []
for c in df_tabuleiro.columns:
    ausentes = df_tabuleiro.filter(col(c).isNull()).count()
    presentes = df_tabuleiro.filter(col(c).isNotNull()).count()
    dados.append((c, presentes, ausentes))

# Cria DataFrame com os resultados
df_resumo = spark.createDataFrame(dados, ["Atributos", "Valores Presentes", "Valores Ausentes"])
df_resumo.show(truncate=False)

+------------------+-----------------+----------------+
|Atributos         |Valores Presentes|Valores Ausentes|
+------------------+-----------------+----------------+
|ID                |9709             |0               |
|Name              |9709             |0               |
|Year Published    |9709             |0               |
|Min Players       |9709             |0               |
|Max Players       |9709             |0               |
|Play Time         |9709             |0               |
|Min Age           |9709             |0               |
|Users Rated       |9709             |0               |
|Rating Average    |9709             |0               |
|BGG Rank          |9709             |0               |
|Complexity Average|9709             |0               |
|Owned Users       |9709             |0               |
|Mechanics         |9709             |0               |
|Domains           |9709             |0               |
+------------------+-----------------+----------

In [0]:
# Obter valores mínimo, máximo e quantidades de valores únicos.

# Colunas que não são numéricas
colunas_excluir = ['Name', 'Mechanics', 'Domains']

# Obtendo valores mínimos, máximos e quantidades de valores únicos.
dados = []
for c in df_tabuleiro.columns:
    uniq_val = df_tabuleiro.select(F.countDistinct(c)).collect()[0][0]
    
    if c in colunas_excluir:
        # Para colunas não numéricas, não calcular min/max
        min_val, max_val = "-", "-"
    else:
        # Para colunas numéricas, calcular min e max
        min_val = df_tabuleiro.agg(F.min(c)).collect()[0][0]
        max_val = df_tabuleiro.agg(F.max(c)).collect()[0][0]
    
    dados.append((c, min_val, max_val, uniq_val))

# Criar DataFrame com os resultados
range_valores = spark.createDataFrame(
    dados, ["Atributos", "Mínimo", "Máximo", "Qtd Valores Únicos"]
)

# Mostrar valores
range_valores.show(truncate=False)

+------------------+------+------+------------------+
|Atributos         |Mínimo|Máximo|Qtd Valores Únicos|
+------------------+------+------+------------------+
|ID                |1     |322289|9709              |
|Name              |-     |-     |9592              |
|Year Published    |-3500 |2021  |162               |
|Min Players       |0     |10    |10                |
|Max Players       |0     |163   |36                |
|Play Time         |0     |60000 |99                |
|Min Age           |0     |21    |19                |
|Users Rated       |30    |102214|2935              |
|Rating Average    |1.43  |9.34  |522               |
|BGG Rank          |1     |20344 |9709              |
|Complexity Average|0.0   |5.0   |377               |
|Owned Users       |3     |155312|3821              |
|Mechanics         |-     |-     |4589              |
|Domains           |-     |-     |39                |
+------------------+------+------+------------------+



#### Tabelas do Modelo Estrela  
_É uma estrutura de banco de dados otimizada para consultas analíticas e relatórios, em contraste com os modelos normalizados (OLTP) otimizados para transações. O seu nome vem da sua forma visual, que se assemelha a uma estrela: uma tabela central (fatos) cercada por várias tabelas auxiliares (dimensões)._  
_Chave Substituta (Surrogate Key) é uma chave artificial, numérica e sequencial criada especificamente para o data warehouse, que substitui as chaves naturais (business keys) dos sistemas operacionais. Criada apenas para conectar tabelas de forma eficiente no data warehouse._

#### Tabelas Dimensão  
_São as "pontas" da estrela. Elas descrevem o contexto dos fatos._  
_Contêm os atributos descritivos e textuais usados para filtrar, agrupar e rotular os dados._  

In [0]:
# Criação Tabela dimensão Jogo
dim_jogo = df_tabuleiro.select(
    col("ID").alias("id_jogo"),
    col("Name").alias("nome_jogo"),
    col("Year Published").alias("ano_publicacao"),
).distinct()

# Adicionar chave substituta
dim_jogo = dim_jogo.withColumn("jogo_sk", monotonically_increasing_id())

# Visualiza amostra da tabela
print("Dimensão Jogo:")
display(dim_jogo.limit(5))

Dimensão Jogo:


id_jogo,nome_jogo,ano_publicacao,jogo_sk
146508,T.I.M.E Stories,2015,0
234477,Battle for Rokugan,2017,1
216091,Unlock!: Escape Adventures - The Formula,2017,2
286096,Tapestry,2019,3
155362,Ca$h 'n Guns (Second Edition),2014,4


In [0]:
# Criação Tabela dimensão Jogabilidade
dim_jogabilidade = df_tabuleiro.select(
    col("ID").alias("id_jogo"),
    col("Min Players").alias("min_jogadores"),
    col("Max Players").alias("max_jogadores"),
    col("Play Time").alias("tempo_jogo"),
    col("Min Age").alias("idade_minima")
).distinct()

# Adicionar chave substituta
dim_jogabilidade = dim_jogabilidade.withColumn("jogabilidade_sk", monotonically_increasing_id())

# Visualiza amostra da tabela
print("Dimensão Jogabilidade:")
display(dim_jogabilidade.limit(5))

Dimensão Jogabilidade:


id_jogo,min_jogadores,max_jogadores,tempo_jogo,idade_minima,jogabilidade_sk
156689,1,5,45,14,0
198773,2,8,15,10,1
2507,3,6,90,14,2
216132,1,4,120,12,3
152162,2,6,30,8,4


In [0]:
# Criação Tabela dimensão mecanicas
dim_mecanicas = df_tabuleiro.select(
    col("ID").alias("id_jogo"),
    explode(split(col("Mechanics"), ", ")).alias("mecanica_jogo")
).distinct()

# Adicionar chave substituta
dim_mecanicas = dim_mecanicas.withColumn("mecanica_sk", monotonically_increasing_id())

# Visualiza amostra da tabela
print("Dimensão Mecânicas:")
display(dim_mecanicas.limit(5))

Dimensão Mecânicas:


id_jogo,mecanica_jogo,mecanica_sk
171623,Income,0
150376,Variable Player Powers,1
54998,Auction: Fixed Placement,2
191862,Set Collection,3
12942,Auction/Bidding,4


In [0]:
# Criação Tabela dimensão Domínios
dim_dominios = df_tabuleiro.select(
    col("ID").alias("id_jogo"),
    explode(split(col("Domains"), ", ")).alias("dominio_jogo")
).distinct()

# Adicionar chave substituta
dim_dominios = dim_dominios.withColumn("dominio_sk", monotonically_increasing_id())

# Visualiza amostra da tabela
print("Dimensão Domínios:")
display(dim_dominios.limit(5))

Dimensão Domínios:


id_jogo,dominio_jogo,dominio_sk
125548,Strategy Games,0
271518,Family Games,1
113873,Wargames,2
220877,Strategy Games,3
235488,Family Games,4


#### Tabelas Fato  
_É o "coração" do modelo. Representa o evento ou processo de negócio que se deseja analisar._  
_Contém as medidas ou métricas numéricas._  
_É composta principalmente por chaves estrangeiras (FK) que se conectam às tabelas de dimensão e pelas medidas em si._

In [0]:
# Criação Tabela Fato Avaliacao
fato_avaliacao = df_tabuleiro.select(
    col("ID").alias("id_jogo"),
    col("Users Rated").alias("usuarios_avaliaram"),
    col("Rating Average").alias("media_avaliacao"),
    col("BGG Rank").alias("classificacao_bgg"),
    col("Complexity Average").alias("media_complexidade"),
    col("Owned Users").alias("usuarios_possuem")
)

# Visualiza amostra da tabela
print("Tabela Fato Avaliação:")
display(fato_avaliacao.limit(5))

Tabela Fato Avaliação:


id_jogo,usuarios_avaliaram,media_avaliacao,classificacao_bgg,media_complexidade,usuarios_possuem
174430,42055,8.79,1,3.86,68323
161936,41643,8.61,2,2.84,65294
224517,19217,8.66,3,3.91,28785
167791,64864,8.43,4,3.24,87099
233078,13468,8.7,5,4.22,16831


#### Salvar as tabelas Delta no Databricks  
_As Tabelas Delta são tabelas que utilizam o formato Delta Lake._  
_Delta Lake é uma camada de armazenamento aberta que traz confiabilidade de transações ACID (Atomicidade, Consistência, Isolamento e Durabilidade) para data lakes._

In [0]:
# Definir o catálogo
spark.sql("USE CATALOG workspace");

# Definir o esquema
spark.sql("USE SCHEMA tabuleiro");

In [0]:
# Criação do modelo estrela no Databricks

print("INICIANDO CRIAÇÃO DO MODELO ESTRELA")
print("-" * 50)

# Lista de tabelas do modelo estrela
tabelas = {
    "tabuleiro.dim_jogo": dim_jogo,
    "tabuleiro.dim_jogabilidade": dim_jogabilidade,
    "tabuleiro.dim_mecanicas": dim_mecanicas,
    "tabuleiro.dim_dominios": dim_dominios,
    "tabuleiro.fato_avaliacao": fato_avaliacao
}

# Apagar e recriar cada tabela
for nome, df in tabelas.items():
    # Apagar tabela Delta, caso exista no esquema 'tabuleiro'
    spark.sql(f"DROP TABLE IF EXISTS {nome}")
    
    # Salvar os dataframe como tabelas Delta no esquema 'tabuleiro'
    df.write.mode("overwrite").saveAsTable(nome)
    print(f"★ {nome} >>> criado")

# Mostrar tabelas criadas
print("\nTabelas criadas no schema 'tabuleiro':")
display(spark.sql("SHOW TABLES IN tabuleiro"))

INICIANDO CRIAÇÃO DO MODELO ESTRELA
--------------------------------------------------
★ tabuleiro.dim_jogo >>> criado
★ tabuleiro.dim_jogabilidade >>> criado
★ tabuleiro.dim_mecanicas >>> criado
★ tabuleiro.dim_dominios >>> criado
★ tabuleiro.fato_avaliacao >>> criado

Tabelas criadas no schema 'tabuleiro':


database,tableName,isTemporary
tabuleiro,dim_dominios,false
tabuleiro,dim_jogabilidade,false
tabuleiro,dim_jogo,false
tabuleiro,dim_mecanicas,false
tabuleiro,fato_avaliacao,false


In [0]:
%sql
DESCRIBE TABLE EXTENDED tabuleiro.dim_jogo;
--DESCRIBE TABLE EXTENDED tabuleiro.dim_jogabilidade;

col_name,data_type,comment
id_jogo,int,ID original do jogo no dataset BoardGameGeek (BGG) (1 a 322289)
nome_jogo,string,Nome completo e oficial do jogo de tabuleiro
ano_publicacao,int,Ano de publicação original do jogo (-3500 a 2021)
jogo_sk,bigint,Chave substituta artificial para a dimensão jogo
,,
# Delta Statistics Columns,,
Column Names,"id_jogo, nome_jogo, ano_publicacao, jogo_sk",
Column Selection Method,first-32,
,,
# Detailed Table Information,,


In [0]:
# Adiciona comentários dos atributos das tabelas do modelo estrela

# --- Comentários para dim_jogo
comentarios_dim_jogo = {
    "jogo_sk": "Chave substituta artificial para a dimensão jogo",
    "id_jogo": "ID original do jogo no dataset BoardGameGeek (BGG) (1 a 322289)",
    "nome_jogo": "Nome completo e oficial do jogo de tabuleiro",
    "ano_publicacao": "Ano de publicação original do jogo (-3500 a 2021)"
}

for coluna, comentario in comentarios_dim_jogo.items():
    spark.sql(f"""
    ALTER TABLE workspace.tabuleiro.dim_jogo 
    ALTER COLUMN {coluna} COMMENT '{comentario}'
    """)

print("Comentários adicionados para dim_jogo")

# --- Comentários para dim_jogabilidade
comentarios_dim_jogabilidade = {
    "jogabilidade_sk": "Chave substituta artificial para a dimensão jogabilidade",
    "id_jogo": "ID original do jogo no dataset BoardGameGeek (BGG) (1 a 322289)",
    "min_jogadores": "Número mínimo de jogadores necessários (0 a 10)",
    "max_jogadores": "Número máximo de jogadores suportados (0 a 163)", 
    "tempo_jogo": "Duração média da partida em minutos (0 a 60000)",
    "idade_minima": "Idade mínima recomendada para jogar (0 a 21)"
}

for coluna, comentario in comentarios_dim_jogabilidade.items():
    spark.sql(f"""
    ALTER TABLE workspace.tabuleiro.dim_jogabilidade 
    ALTER COLUMN {coluna} COMMENT '{comentario}'
    """)

print("Comentários adicionados para dim_jogabilidade")

# --- Comentários para dim_mecanicas
comentarios_dim_mecanicas = {
    "mecanica_sk": "Chave substituta artificial para a dimensão mecânicas",
    "id_jogo": "ID original do jogo no dataset BoardGameGeek (BGG) (1 a 322289)",
    "mecanica_jogo": "Regras e dinâmica de jogabilidade."
}

for coluna, comentario in comentarios_dim_mecanicas.items():
    spark.sql(f"""
    ALTER TABLE workspace.tabuleiro.dim_mecanicas 
    ALTER COLUMN {coluna} COMMENT '{comentario}'
    """)

print("Comentários adicionados para dim_mecanicas")

# --- Comentários para dim_dominios
comentarios_dim_dominios = {
    "dominio_sk": "Chave substituta artificial para a dimensão domínios", 
    "id_jogo": "ID original do jogo no dataset BoardGameGeek (BGG) (1 a 322289)",
    "dominio_jogo": "Categorias temáticas ou gêneros dos jogos."
}

for coluna, comentario in comentarios_dim_dominios.items():
    spark.sql(f"""
    ALTER TABLE workspace.tabuleiro.dim_dominios 
    ALTER COLUMN {coluna} COMMENT '{comentario}'
    """)

print("Comentários adicionados para dim_dominios")

# --- Comentários para fato_avaliacao
comentarios_fato_avaliacao = {
    "id_jogo": "ID original do jogo no dataset BoardGameGeek (BGG) (1 a 322289)",
    "usuarios_avaliaram": "Quantidade de usuários que avaliaram o jogo (30 a 102214)",
    "media_avaliacao": "Nota média recebida (1.43 a 9.34)",
    "classificacao_bgg": "Posição no ranking da BoardGameGeek (1 a 20344)",
    "media_complexidade": "Nível médio de dificuldade/complexidade (0 a 5.0)",
    "usuarios_possuem": "Quantidade de usuários que possuem o jogo (0 a 155312)"
}

for coluna, comentario in comentarios_fato_avaliacao.items():
    spark.sql(f"""
    ALTER TABLE workspace.tabuleiro.fato_avaliacao 
    ALTER COLUMN {coluna} COMMENT '{comentario}'
    """)

print("Comentários adicionados para fato_avaliacao")

Comentários adicionados para dim_jogo
Comentários adicionados para dim_jogabilidade
Comentários adicionados para dim_mecanicas
Comentários adicionados para dim_dominios
Comentários adicionados para fato_avaliacao


In [0]:
# Adiciona etiquetas (tags) dos atributos das tabelas do modelo estrela

# --- Etiquetas para dim_jogo
etiquetas_dim_jogo = {
    "id_jogo": "identificador",
    "nome_jogo": "descricao",
    "ano_publicacao": "temporal",
    "jogo_sk": "chave_substituta"
}

for coluna, etiqueta in etiquetas_dim_jogo.items():
    spark.sql(f"""
    ALTER TABLE workspace.tabuleiro.dim_jogo 
    ALTER COLUMN {coluna} SET TAGS ('{etiqueta}')
    """)

print("Etiquetas adicionadas para dim_jogo")

# --- Etiquetas para dim_jogabilidade
etiquetas_dim_jogabilidade = {
    "jogabilidade_sk": "chave_substituta",
    "id_jogo": "identificador",
    "min_jogadores": "jogabilidade_min",
    "max_jogadores": "jogabilidade_max",
    "tempo_jogo": "duracao",
    "idade_minima": "faixa_etaria"
}

for coluna, etiqueta in etiquetas_dim_jogabilidade.items():
    spark.sql(f"""
    ALTER TABLE workspace.tabuleiro.dim_jogabilidade 
    ALTER COLUMN {coluna} SET TAGS ('{etiqueta}')
    """)

print("Etiquetas adicionadas para dim_jogabilidade")

# --- Etiquetas para dim_mecanicas
etiquetas_dim_mecanicas = {
    "mecanica_sk": "chave_substituta",
    "id_jogo": "identificador",
    "mecanica_jogo": "dinamica"
}

for coluna, etiqueta in etiquetas_dim_mecanicas.items():
    spark.sql(f"""
    ALTER TABLE workspace.tabuleiro.dim_mecanicas 
    ALTER COLUMN {coluna} SET TAGS ('{etiqueta}')
    """)

print("Etiquetas adicionadas para dim_mecanicas")

# --- Etiquetas para dim_dominios
etiquetas_dim_dominios = {
    "dominio_sk": "chave_substituta",
    "id_jogo": "identificador",
    "dominio_jogo": "categoria"
}

for coluna, etiqueta in etiquetas_dim_dominios.items():
    spark.sql(f"""
    ALTER TABLE workspace.tabuleiro.dim_dominios 
    ALTER COLUMN {coluna} SET TAGS ('{etiqueta}')
    """)

print("Etiquetas adicionadas para dim_dominios")

# --- Etiquetas para fato_avaliacao
etiquetas_fato_avaliacao = {
    "id_jogo": "identificador",
    "usuarios_avaliaram": "engajamento",
    "media_avaliacao": "desempenho",
    "classificacao_bgg": "ranking",
    "media_complexidade": "nivel",
    "usuarios_possuem": "popularidade"
}

for coluna, etiqueta in etiquetas_fato_avaliacao.items():
    spark.sql(f"""
    ALTER TABLE workspace.tabuleiro.fato_avaliacao 
    ALTER COLUMN {coluna} SET TAGS ('{etiqueta}')
    """)

print("Etiquetas adicionadas para fato_avaliacao")

Etiquetas adicionadas para dim_jogo
Etiquetas adicionadas para dim_jogabilidade
Etiquetas adicionadas para dim_mecanicas
Etiquetas adicionadas para dim_dominios
Etiquetas adicionadas para fato_avaliacao


In [0]:
# Mostra qual catálogo sua sessão está usando no momento
spark.sql("SELECT current_catalog() AS Catalogo, current_schema() AS Esquema").show()

+---------+---------+
| Catalogo|  Esquema|
+---------+---------+
|workspace|tabuleiro|
+---------+---------+



In [0]:
print("TABELA: dim_jogo")
spark.sql("DESCRIBE TABLE dim_jogo").show(truncate=False)
print("TABELA: dim_jogabilidade")
spark.sql("DESCRIBE TABLE dim_jogabilidade").show(truncate=False)
print("TABELA: dim_dominios")
spark.sql("DESCRIBE TABLE dim_dominios").show(truncate=False)
print("TABELA: dim_mecanicas")
spark.sql("DESCRIBE TABLE dim_mecanicas").show(truncate=False)
print("TABELA: fato_avaliacao")
spark.sql("DESCRIBE TABLE fato_avaliacao").show(truncate=False)


TABELA: dim_jogo
+--------------+---------+---------------------------------------------------------------+
|col_name      |data_type|comment                                                        |
+--------------+---------+---------------------------------------------------------------+
|id_jogo       |int      |ID original do jogo no dataset BoardGameGeek (BGG) (1 a 322289)|
|nome_jogo     |string   |Nome completo e oficial do jogo de tabuleiro                   |
|ano_publicacao|int      |Ano de publicação original do jogo (-3500 a 2021)              |
|jogo_sk       |bigint   |Chave substituta artificial para a dimensão jogo               |
+--------------+---------+---------------------------------------------------------------+

TABELA: dim_jogabilidade
+---------------+---------+---------------------------------------------------------------+
|col_name       |data_type|comment                                                        |
+---------------+---------+------------------